In [30]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import torch
from helper import *
from coval.coval.conll.reader import get_coref_infos
from coval.coval.eval.evaluator import evaluate_documents as evaluate
from coval.coval.eval.evaluator import muc, b_cubed, ceafe, lea
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False


ModuleNotFoundError: No module named 'helper'

In [ ]:
def read(key, response):
    '''read keyfile 
        key:  the keyfile path for gold_map
        response: the keyfile path for model_map  
    '''
    return get_coref_infos('%s' % key, '%s' % response,
            False, False, True)

def give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path):
    '''get MUC, B-cubed, CEAFe, LEA, CoNLL F1 evaluation
        gold_map_path: the path for gold_map
        model_map_path: the path for model_map
        keyfile_save_path: the path to save the keyfiles generated by the func 'read'
    '''
    gold_map=pd.read_pickle(gold_map_path)
    model_map=pd.read_pickle(model_map_path)
    save_folder=os.path.join(keyfile_save_path)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        print(f"Folder '{keyfile_save_path}' created successfully")  
    else:
        print('Testing') 
    gold_map_conll_file=save_folder + f'/evt_gold_test.keyfile'
    model_map_conll_file=save_folder + f'/evt_model_test.keyfile'
    generate_key_file(gold_map.items(), 'evt', save_folder, gold_map_conll_file)
    generate_key_file(model_map.items(), 'evt', save_folder, model_map_conll_file)
    doc= read(gold_map_conll_file, model_map_conll_file)
    mr, mp, mf = np.round(np.round(evaluate(doc, muc), 3) * 100, 1)
    br, bp, bf = np.round(np.round(evaluate(doc, b_cubed), 3) * 100, 1)
    cr, cp, cf = np.round(np.round(evaluate(doc, ceafe), 3) * 100, 1)
    lr, lp, lf = np.round(np.round(evaluate(doc, lea), 3) * 100, 1)
    conf = np.round((mf + bf + cf) / 3, 1)
    muc_res = f"MUC: R {mr} P {mp} F1 {mf}\n"
    bcubed_res = f"B-cubed: R {br} P {bp} F1 {bf}\n"
    ceafe_res = f"CEAFe: R {cr} P {cp} F1 {cf}\n"
    lea_res = f"LEA: R {lr} P {lp} F1 {lf}\n"
    conll_res = f"CoNLL F1: {conf}\n"
    multi_metrics_str = muc_res + bcubed_res + ceafe_res + lea_res + conll_res
    print(multi_metrics_str)
    with open(os.path.join(save_folder,'multi_metrics.txt'), "w") as file:
        file.write(multi_metrics_str)
    multi_metrics_dict = {'MUC':{'R':mr, 'P':mp, 'F1': mf},
                    'B-cubed':{'R':br, 'P':bp, 'F1': bf},
                    'CEAFe':{'R':cr, 'P':cp, 'F1': cf},
                    'LEA':{'R',lr, 'P',lp,'F1',lf},
                    'CoNLL F1':{conf}}
    return multi_metrics_dict

In [ ]:
def give_evaluation_result_2(gold_map_path,model_map_path,save_path):
    gold_map=pd.read_pickle(gold_map_path)
    model_map=pd.read_pickle(model_map_path)
    save_folder=save_path
    # if not os.path.exists(save_folder):
    #     os.makedirs(save_folder)
    #     print(f"Folder '{model_tag}' created successfully")  
    # else:
    print('Testing') 
    gold_map_conll_file=save_folder + f'/evt_gold_test.keyfile'
    model_map_conll_file=save_folder + f'/evt_model_test.keyfile'
    generate_key_file(gold_map.items(), 'evt', save_folder, gold_map_conll_file)
    generate_key_file(model_map.items(), 'evt', save_folder, model_map_conll_file)
    doc= read(gold_map_conll_file, model_map_conll_file)
    mr, mp, mf = np.round(np.round(evaluate(doc, muc), 3) * 100, 1)
    br, bp, bf = np.round(np.round(evaluate(doc, b_cubed), 3) * 100, 1)
    cr, cp, cf = np.round(np.round(evaluate(doc, ceafe), 3) * 100, 1)
    lr, lp, lf = np.round(np.round(evaluate(doc, lea), 3) * 100, 1)
    conf = np.round((mf + bf + cf) / 3, 1)
    mu=(mr,mp,mf)
    b_cu=(br,bp,bf)
    cea=(cr,cp,cf)
    le=(lr,lp,lf)
    return mu, b_cu,cea,le,conf


def B_cubed_pairwise_t_test(test_programme_path):
    base_path=os.path.join(test_programme_path,'base')
    aug_path=os.path.join(test_programme_path,'aug')
    base_b_cubed_list=[]
    aug_b_cubed_list=[]
    for t in range(10):
        base_turn_path=os.path.join(base_path,'turn_{}'.format(t))
        aug_turn_path=os.path.join(aug_path,'turn_{}'.format(t))
        base_mu,base_b_cu,base_cea,base_le,base_conf=give_evaluation_result_2(base_turn_path+r'/gold_map',base_turn_path+r'/model_map',base_turn_path) 
        aug_mu,aug_b_cu,aug_cea,aug_le,aug_conf=give_evaluation_result_2(aug_turn_path+r'/gold_map',aug_turn_path+r'/model_map',aug_turn_path) 
        base_b_cubed_list.append(base_b_cu[-1])
        aug_b_cubed_list.append(aug_b_cu[-1])
    print(stats.ttest_rel(base_b_cubed_list, aug_b_cubed_list))
# ttest for ecb+
stats.ttest_rel([84.1, 85.6, 83.5, 84.4, 84.3, 84.5,82.8], [85.5, 86, 85.1, 84.9, 84.8, 84.9, 84.3])

# 1. Main Exp

## 1.1 ECB+

In [ ]:
exp_type = 'main'
dataset = 'ecb'
ECRsystem = 'baseline'
save_path_prefix =  f'/home/yaolong/Rationale4CDECR-main/outputs/{exp_type}/{dataset}/{ECRsystem}/dual/dual_save_data/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
exp_type = 'main'
dataset = 'ecb'
ECRsystem = 'enhanced'
save_path_prefix =  f'/home/yaolong/Rationale4CDECR-main/outputs/{exp_type}/{dataset}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 1.2 FCC

In [ ]:
exp_type = 'main'
dataset = 'fcc'
ECRsystem = 'baseline'
save_path_prefix =  f'../outputs/{exp_type}/{dataset}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
exp_type = 'main'
dataset = 'fcc'
ECRsystem = 'enhanced'
save_path_prefix =  f'../outputs/{exp_type}/{dataset}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 1.3 GVC

In [ ]:
exp_type = 'main'
dataset = 'gvc'
ECRsystem = 'baseline'
save_path_prefix =  f'../outputs/{exp_type}/{dataset}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
exp_type = 'main'
dataset = 'gvc'
ECRsystem = 'enhanced'
save_path_prefix =  f'../outputs/{exp_type}/{dataset}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

# 2. Ablation Study

## 2.1 CAD

In [ ]:
exp_type = 'ablation_study'
data_type = 'CAD'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 2.2 TIA

In [ ]:
exp_type = 'ablation_study'
data_type = 'TIA'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 2.3 CIA

In [ ]:
exp_type = 'ablation_study'
data_type = 'CIA'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 2.4 TAD

In [ ]:
exp_type = 'ablation_study'
data_type = 'TAD'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 2.5 RMCT

In [ ]:
exp_type = 'ablation_study'
data_type = 'RMCT'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 3. Out-Of-the-Domain Test

## 3.1 ECB_Cross_FCC

In [ ]:
exp_type = 'ood_test'
data_type = 'ecb_cross_fcc'
ECRsystem = 'baseline'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
exp_type = 'ood_test'
data_type = 'ecb_cross_fcc'
ECRsystem = 'enhanced'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

## 3.2 FCC_Cross_ECB

In [ ]:
exp_type = 'ood_test'
data_type = 'fcc_cross_ecb'
ECRsystem = 'baseline'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
exp_type = 'ood_test'
data_type = 'fcc_cross_ecb'
ECRsystem = 'enhanced'
save_path_prefix =  f'../outputs/{exp_type}/{data_type}/{ECRsystem}/eval_results'
gold_map_path = os.path.join(save_path_prefix ,'gold_map') 
model_map_path = os.path.join(save_path_prefix ,'model_map') 
keyfile_save_path = os.path.join(save_path_prefix ,'multi_metrics')
ecb_main_enhanced = give_evaluation_result(gold_map_path, model_map_path, keyfile_save_path)

In [ ]:
# ttest for ecb+
stats.ttest_rel([84.1, 85.6, 83.5, 84.4, 84.3, 84.5,82.8], [85.5, 86, 85.1, 84.9, 84.8, 84.9, 84.3])